In [1]:
import azureml.core

print("Azure Machine Learning SDK Version:", azureml.core.VERSION)

Azure Machine Learning SDK Version: 1.37.0


In [2]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, sep = ' | ')

ml-workspace | eastus | ml-rg


In [3]:
from azureml.core.compute import AmlCompute, ComputeTarget
import os

# Choose a name and maximum size for your cluster
compute_name = "cpu-cluster-d2"
compute_min_nodes = 0
compute_max_nodes = 4
vm_size = "STANDARD_D2_V2"

if compute_name in ws.compute_targets:
    print("Found an existing compute target of name: " + compute_name)
    compute_target = ws.compute_targets[compute_name]
    # Note: you may want to make sure compute_target is of type AmlCompute        
else:
    print("Creating new compute target...")
    provisioning_config = AmlCompute.provisioning_configuration(
        vm_size=vm_size,
        min_nodes=compute_min_nodes, 
        max_nodes=compute_max_nodes)
        
    # Create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

print(compute_target.get_status().serialize())

Found an existing compute target of name: cpu-cluster-d2
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2022-03-07T22:21:22.300000+00:00', 'errors': None, 'creationTime': '2022-01-15T14:45:39.283011+00:00', 'modifiedTime': '2022-01-15T14:45:42.951037+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [4]:
from azureml.core.experiment import Experiment

from files import BCTrade_utils
#env ='Btc_with_indicators'
env ='Bitstamp_1'
experiment_name = BCTrade_utils.registerBCTrade(env)
exp = Experiment(workspace=ws, name=experiment_name)

In [5]:
from 	azureml.data.data_reference import DataReference
from azureml.core import Dataset

datastore = ws.get_default_datastore()

path='checkpoints_'
path+='BCTrade-Bitstamp_1_1646556835_d8cf9986_head'
checkpoint_ds= Dataset.File.from_files(DataReference(datastore, path_on_datastore=path))

checkpoint = '-351'


In [6]:
from azureml.contrib.train.rl import ReinforcementLearningEstimator, Ray
from azureml.core.environment import Environment

training_algorithm = "PPO"
rl_environment = env
video_capture = False

algorithm_config = '\'{"num_gpus": 0, "num_workers": 1}\''
#if video_capture:
#    algorithm_config = '\'{"num_gpus": 0, "num_workers": 1, "monitor": true}\''
#else:
#    algorithm_config = '\'{"num_gpus": 0, "num_workers": 1, "monitor": false}\''
script_params = {

    # Training algorithm
    "--run": training_algorithm,
    
    # Training environment
    "--env": '\'{"name": "'+rl_environment+'", "singlerun": True, "spreadfactor":0.0}\'',
    #"--env": '\'{"name": "'+rl_environment+'", "checkpoint":"'+checkpoint+'", "singlerun":False, "spreadfactor":0.001}\'',

    #"--spreadfactor": 0.1,
    
    # Algorithm-specific parameters
    "--config": algorithm_config,
    
    # Stop conditions
    #"--stop": '\'{"time_total_s": 25000}\'',
    "--stop": '\'{"time_total_s": 15000,"episode_reward_mean":5}\'',
    #"--stop": '\'{"time_total_s": 36000,"episode_reward_mean":2}\'',
    #"--stop": '\'{"time_total_s": 60000}\'',
    
    # Frequency of taking checkpoints
    "--checkpoint-freq": 1,
    
    # If a checkpoint should be taken at the end - optional argument with no value
    "--checkpoint-at-end": "",
    
    # Log directory
    "--local-dir": './logs'
}

xvfb_env = None
#if video_capture:
    # Ray's video capture support requires to run everything under a headless display driver called (xvfb).
    # There are two parts to this:
    # 1. Use a custom docker file with proper instructions to install xvfb, ffmpeg, python-opengl
    # and other dependencies.
   
#    with open("files/docker/Dockerfile", "r") as f:
#        dockerfile=f.read()

#    xvfb_env = Environment(name='xvfb-vdisplay')
#    xvfb_env.docker.base_image = None
#    xvfb_env.docker.base_dockerfile = dockerfile
    
    # 2.  Execute the Python process via the xvfb-run command to set up the headless display driver.
#    xvfb_env.python.user_managed_dependencies = True
#    xvfb_env.python.interpreter_path = "xvfb-run -s '-screen 0 640x480x16 -ac +extension GLX +render' python"


training_estimator = ReinforcementLearningEstimator(

    # Location of source files
    source_directory='files',
    
    # Python script file
    entry_script='BCTrade_training.py',
    
    # A dictionary of arguments to pass to the training script specified in ``entry_script``
    script_params=script_params,
    
    # The Azure Machine Learning compute target set up for Ray head nodes
    compute_target=compute_target,
    
    
    inputs=[
        checkpoint_ds.as_named_input('artifacts_dataset'),
        checkpoint_ds.as_named_input('artifacts_path').as_mount()],
    
    # Reinforcement learning framework. Currently must be Ray.
    rl_framework=Ray(),
    
    # Custom environmnet for Xvfb
    environment=xvfb_env
)

In [7]:
training_run = exp.submit(training_estimator)

In [8]:
from azureml.widgets import RunDetails

RunDetails(training_run).show()

_RLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 'sdk_v…